In [48]:
#Import definitions as new module
import sys
import os
# sys.path.insert(0, os.path.dirname(os.path.abspath('.')))
os.chdir(r'C:\Users\Pia\OneDrive\Data Science\experiment_design\ex_2\CoE_dataset')
import pandas as pd
pd.options.display.max_columns = 102
pd.options.display.max_rows = 100
import numpy as np
from datetime import datetime
import csv
import xml.etree.ElementTree as et
from pathlib import Path
import definitions as defs
import aggregate_to_csv as agg
from random import seed, sample, choice

try:
    test = defs.MODE_TEST
    test2 = agg.get_parameters.__name__
    print('modules imported successfully')
except:
    print('ERR')

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

modules imported successfully


In [76]:
def load_data(data_set):

    # get the columnnames:
    xtree = et.parse(r'Dev_Set\XML\A_Fish_Called_Wanda.xml')
    xroot = xtree.getroot()
    columns = list(xroot.find('movie').keys())
    df = pd.DataFrame(columns = columns)

    for movie_name in os.listdir(data_set + '_Set/XML'):  
        xtree = et.parse(data_set + '_Set/XML/' + movie_name)
        xroot = xtree.getroot()
        instance = []
        for element in columns:
            if xroot[0] is not None: # and element != 'goodforairplane': # feature "good for airplane" is not filled in 
                instance.append(xroot.find('movie').get(element))
            else:
                pass
        if len(instance) < len(columns):
            instance.append(movie_name[:-3])
        my_series = pd.Series(instance, index = columns)
        df = df.append(my_series, ignore_index=True)

    df.set_index('title', inplace=True)

    df = df.loc[:, ['language', 'year', 'genre', 'country', 'runtime', 'rated', 'metascore', 'imdbRating', 'tomatoUserRating']]

    # make the feature 'runtime' numeric:
    df.runtime = df.runtime.apply(lambda x: x[:-4])
    df.runtime = pd.to_numeric(df.runtime, errors='coerce')

    # bring the entries of 'rated' which were not filled out into a unique shape:
    df.rated.replace(['NOT RATED', 'UNRATED'], 'N/A', inplace=True)

    for feature in ['year', 'metascore', 'imdbRating', 'tomatoUserRating']:
        df.loc[:, feature] = pd.to_numeric(df.loc[:, feature] , errors='coerce')

    return df


def get_dummies(df, selected_features=['language', 'genre', 'country', 'rated']):
    
    df_tmp = df.copy()
    for feature in set(selected_features).intersection(['language', 'genre', 'country', 'rated']):  # we only get dummy 
                                                                                                     # variables for categorical
                                                                                                      # data
        # split the variables with various entries: 
        one_hot = df.copy().loc[:, feature].str.split(', ', expand=True).stack()
        
        one_hot = pd.get_dummies(one_hot, prefix=feature, drop_first=True).groupby(level=0).sum()
        df_tmp = df_tmp.drop(feature, axis=1)
        df_tmp = df_tmp.merge(one_hot, left_index=True, right_index=True)
    
    return df_tmp

In [92]:
df_dev = load_data('Dev')
labels = pd.read_excel(r'Dev_Set\dev_set_groundtruth_and_trailers.xls', index_col=0).goodforairplane
df_dev = df_dev.merge(labels, left_index=True, right_index=True)

df_test = load_data('Test')
labels = pd.read_csv(r'Test_Set\CoeTestLabels.csv', index_col=0).goodforairplanes
df_test = df_test.merge(labels, left_index=True, right_index=True)
df_test.rename(columns={'goodforairplanes': 'goodforairplane'}, inplace=True)

df_dev_test = df_dev.append(df_test)
df_dev_test = get_dummies(df_dev_test)
df_dev_test.index = df_dev_test.index.str.replace('\'', '')


df_dev = df_dev_test.iloc[:86]
df_dev.to_csv('metadata_ratings_dev.csv', index=True)

df_test = df_dev_test.iloc[86:]
df_test.to_csv('metadata_ratings_test.csv', index=True)

Index(['title', 'year', 'rated', 'released', 'runtime', 'genre', 'director',
       'writer', 'actors', 'plot', 'language', 'country', 'awards', 'poster',
       'metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'type', 'tomatoMeter',
       'tomatoImage', 'tomatoRating', 'tomatoReviews', 'tomatoFresh',
       'tomatoRotten', 'tomatoConsensus', 'tomatoUserMeter',
       'tomatoUserRating', 'tomatoUserReviews', 'DVD', 'BoxOffice',
       'Production', 'Website'],
      dtype='object')
Index(['title', 'year', 'rated', 'released', 'runtime', 'genre', 'director',
       'writer', 'actors', 'plot', 'language', 'country', 'awards', 'poster',
       'metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'type', 'tomatoMeter',
       'tomatoImage', 'tomatoRating', 'tomatoReviews', 'tomatoFresh',
       'tomatoRotten', 'tomatoConsensus', 'tomatoUserMeter',
       'tomatoUserRating', 'tomatoUserReviews', 'DVD', 'BoxOffice',
       'Production', 'Website'],
      dtype='object')


In [94]:
ratings = ['metascore', 'imdbRating', 'tomatoUserRating']

metadata_dev = df_dev[df_dev.columns.difference(ratings)]
metadata_dev.to_csv('metadata_dev.csv', index=True)

metadata_test = df_test[df_test.columns.difference(ratings)]
metadata_test.to_csv('metadata_test.csv', index=True)